In [1]:
!pip install langchain langchain-openai langchain-pinecone langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 14.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import os

os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [3]:
os.environ['OPENAI_EMBEDDING_MODEL'] = userdata.get('OPENAI_EMBEDDING_MODEL')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

## API -> 문서

In [19]:
import requests
from langchain_core.documents import Document

def json_to_documents(api_json: dict) -> list[Document]:
    documents = []

    for entry in api_json['body']['items']:
        data = entry['item']

        # Helper function to get and strip a value, handling None
        def get_and_strip(data_dict, key):
            value = data_dict.get(key)
            return value.strip() if isinstance(value, str) else ''

        # page_content 구성
        text = f"""
제품명: {get_and_strip(data, 'PRDUCT')}
제조사: {get_and_strip(data, 'ENTRPS')}
기능성: {get_and_strip(data, 'MAIN_FNCTN')}
섭취 시 주의사항: {get_and_strip(data, 'INTAKE_HINT1')}
보관조건: {get_and_strip(data, 'PRSRV_PD')}
유통기한: {get_and_strip(data, 'DISTB_PD')}
"""

        # metadata 구성
        metadata = {
            "등록일자": data.get("STTEMNT_NO"),
            "제조사": data.get("ENTRPS"),
            "기준규격": get_and_strip(data, "BASE_STANDARD")
        }

        # Document 생성
        documents.append(Document(page_content=text, metadata=metadata))

    return documents


def fetch_all_documents(api_url, api_key, num_of_rows=100) -> list[Document]:
    all_documents = []

    # 먼저 1페이지 호출
    params = {
        "ServiceKey": api_key,
        "pageNo": "1",
        "numOfRows": str(num_of_rows),
        "type": "json",
    }

    response = requests.get(api_url, params=params, timeout=10)
    response.raise_for_status()
    first_page = response.json()

    total_count = int(first_page['body']['totalCount'])
    total_pages = (total_count // num_of_rows) + (1 if total_count % num_of_rows else 0)

    # 1페이지 → Document 추출
    all_documents.extend(json_to_documents(first_page))

    # 2페이지부터 마지막 페이지까지 반복
    for page in range(2, total_pages + 1):
        params['pageNo'] = str(page)
        response = requests.get(api_url, params=params, timeout=10)
        response.raise_for_status()
        page_json = response.json()

        docs = json_to_documents(page_json)
        all_documents.extend(docs)
        print(f"📄 {page}/{total_pages} 페이지 수집 완료")

    print(f"\n✅ 총 {len(all_documents)}개의 Document 객체 생성 완료")
    return all_documents

In [20]:
url = 'http://apis.data.go.kr/1471000/HtfsInfoService03/getHtfsItem01'
api_key = 'nsfVX4dKQRFTeyldmuRefFQqL8xOsDkkyw8TsU4dA4fO9vq7Zl7JTbrakHnVYBqRG62CWBhhOVwBaGgCBbm3AA=='

documents = fetch_all_documents(url, api_key)
print(documents[0].page_content)
print(documents[0].metadata)

📄 2/418 페이지 수집 완료
📄 3/418 페이지 수집 완료
📄 4/418 페이지 수집 완료
📄 5/418 페이지 수집 완료
📄 6/418 페이지 수집 완료
📄 7/418 페이지 수집 완료
📄 8/418 페이지 수집 완료
📄 9/418 페이지 수집 완료
📄 10/418 페이지 수집 완료
📄 11/418 페이지 수집 완료
📄 12/418 페이지 수집 완료
📄 13/418 페이지 수집 완료
📄 14/418 페이지 수집 완료
📄 15/418 페이지 수집 완료
📄 16/418 페이지 수집 완료
📄 17/418 페이지 수집 완료
📄 18/418 페이지 수집 완료
📄 19/418 페이지 수집 완료
📄 20/418 페이지 수집 완료
📄 21/418 페이지 수집 완료
📄 22/418 페이지 수집 완료
📄 23/418 페이지 수집 완료
📄 24/418 페이지 수집 완료
📄 25/418 페이지 수집 완료
📄 26/418 페이지 수집 완료
📄 27/418 페이지 수집 완료
📄 28/418 페이지 수집 완료
📄 29/418 페이지 수집 완료
📄 30/418 페이지 수집 완료
📄 31/418 페이지 수집 완료
📄 32/418 페이지 수집 완료
📄 33/418 페이지 수집 완료
📄 34/418 페이지 수집 완료
📄 35/418 페이지 수집 완료
📄 36/418 페이지 수집 완료
📄 37/418 페이지 수집 완료
📄 38/418 페이지 수집 완료
📄 39/418 페이지 수집 완료
📄 40/418 페이지 수집 완료
📄 41/418 페이지 수집 완료
📄 42/418 페이지 수집 완료
📄 43/418 페이지 수집 완료
📄 44/418 페이지 수집 완료
📄 45/418 페이지 수집 완료
📄 46/418 페이지 수집 완료
📄 47/418 페이지 수집 완료
📄 48/418 페이지 수집 완료
📄 49/418 페이지 수집 완료
📄 50/418 페이지 수집 완료
📄 51/418 페이지 수집 완료
📄 52/418 페이지 수집 완료
📄 53/418 페이지 수집 완료
📄 54/418 페이지 수집 완료
📄

In [21]:
print(documents[41000].page_content)
print(documents[41000].metadata)


제품명: 홍트리바이오틱스
제조사: 주식회사한미양행
기능성: [홍삼] 혈소판 응집억제를 통한 혈액흐름·기억력 개선·항산화에 도움을 줄 수 있음.
[옥타코사놀함유유지] 지구력 증진에 도움을 줄 수 있음.
[프락토올리고당] 장내 유익균 증식 및 배변활동 원활에 도움을 줄 수 있음.
섭취 시 주의사항: 의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의
섭취 시 가스참, 트림, 복통, 복부팽만감 등이 발생할 수 있음.
이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
알레르기 체질이신 경우 성분을 확인 한 후 섭취하시기 바랍니다.
보관조건: 고온다습한 곳이나 직사광선을 피하여 서늘한 곳에 보관.
유통기한: 24개월

{'등록일자': '200400150831317', '제조사': '주식회사한미양행', '기준규격': '(1) 성상 : 이미, 이취가 없으며 고유의 향미를 지닌 흑갈색의 편상\n(2) 진세노사이드 Rg1, Rb1 및 Rg3의 합 : 표시량(2.5 mg/10 g)의 80% 이상\n(3) 옥타코사놀 : 표시량(12 mg/10 g)의 80% 이상 120% 이하\n(4) 프락토올리고당 : 표시량(3,135 mg/10 g)의 80% 이상 120% 이하\n(5) 납(mg/kg) : 1.0 이하\n(6) 대장균군 : 음성'}


## 문서 -> 벡터스토어

In [22]:
# 문서로부터 벡터스토어 생성
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone
import time
import os


# 임베딩 모델
embeddings = OpenAIEmbeddings(model=os.environ['OPENAI_EMBEDDING_MODEL'])

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(documents)

# Pinecone 생성
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

index_name = 'health-supplement-rag'

# index 만들어논거 없으면 새로 생성
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=embeddings.get_dimension(),
        metric='cosine'
    )
    #
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

vector_store = PineconeVectorStore(index_name=index_name, embedding=embeddings)

# 배치로 밀어 넣기
batch_size = 100  # 배치사이즈 지정
for i in range(0, len(split_documents), batch_size):
    batch = split_documents[i:i + batch_size]
    # vector_store.add_documents(batch)
    print(f"Added batch {i//batch_size + 1}/{(len(split_documents)//batch_size) + 1}")

print("\n✅ All documents added to Pinecone vector store.")

Added batch 1/433
Added batch 2/433
Added batch 3/433
Added batch 4/433
Added batch 5/433
Added batch 6/433
Added batch 7/433
Added batch 8/433
Added batch 9/433
Added batch 10/433
Added batch 11/433
Added batch 12/433
Added batch 13/433
Added batch 14/433
Added batch 15/433
Added batch 16/433
Added batch 17/433
Added batch 18/433
Added batch 19/433
Added batch 20/433
Added batch 21/433
Added batch 22/433
Added batch 23/433
Added batch 24/433
Added batch 25/433
Added batch 26/433
Added batch 27/433
Added batch 28/433
Added batch 29/433
Added batch 30/433
Added batch 31/433
Added batch 32/433
Added batch 33/433
Added batch 34/433
Added batch 35/433
Added batch 36/433
Added batch 37/433
Added batch 38/433
Added batch 39/433
Added batch 40/433
Added batch 41/433
Added batch 42/433
Added batch 43/433
Added batch 44/433
Added batch 45/433
Added batch 46/433
Added batch 47/433
Added batch 48/433
Added batch 49/433
Added batch 50/433
Added batch 51/433
Added batch 52/433
Added batch 53/433
Ad

In [25]:
# 존재하는 인덱스에 접근/검색
from pprint import pprint

retriever = vector_store.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 3} # 유사한 문서 3개까지 검색
)
pprint(retriever.invoke('피로개선에 도움이 되는 영양제는?'))

[Document(id='8609cb9f-665e-4356-bfde-f54f23470036', metadata={'기준규격': '1) 성상 : 이미, 이취가 없고 고유의 향미가 있는 갈색의 장방형 제피정제 \n2) 로사빈 : 표시량(6 mg /1,000 mg)의 80~120% \n3) 나이아신 : 표시량(7.5 mgNE /1,000 mg)의 80~150%\n4) 납(mg/kg) : 1.0 이하 \n5) 카드뮴(mg/kg) : 0.5 이하 \n6) 총수은(mg/Kg) : 0.5 이하 \n7) 총비소(mg/Kg) : 1.0 이하 \n8) 대장균군 : 음성 \n9) 붕해시험 : 적합', '등록일자': '20040015107744', '제조사': '(주)유유헬스케어'}, page_content='제품명: 피로개선에 도움을 줄 수 있는 홍경천\n제조사: (주)유유헬스케어\n기능성: 스트레스로 인한 피로개선에 도움을 줄 수 있음.\n\n①체내 에너지 생성에 필요\n섭취 시 주의사항: 특정원료에 알레르기가 있거나 질병치료, 약물투여 중인 분은 섭취 전 전문가와 상의하십시오.  \n제품 개봉시 포장재에 의해 상처를 입을 수 있으니 주의하십시오.\n보관조건: 직사광선 및 고온다습한 곳을 피해 서늘한 곳에 보관하십시오.\n유통기한: 제조일로부터 24개월까지'),
 Document(id='0c0a6382-6cbc-4971-af77-52a8bce0caea', metadata={'기준규격': '성상 : 이미, 이취가 없고 고유의 향미가 있는 갈색의 코팅정제\r\n진세노사이드 Rb1,Rg1 및 Rg3의 합 : 80%이상 (표시량:3mg/1,660mg)\r\n비타민A : 표시량의 80~150% (표시량:485ugRE/1,660mg)\r\n비타민B1 : 표시량의 80~180% (표시량:16mg/1,660mg)\r\n비타민B2 : 표시량의 80~180% (표시량:1.5mg/1,660mg)\r\n비타민B6 : 표시량의 80~150% (표시량:16.5mg/1,660mg)\r\n비타민B12 : 

## RAG 구성 (Retriever + LLM → 최종 답변 생성) 단계

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4.1-mini",  #
    temperature=0.3, # 0.3 답변이 신중하고 일관
    max_tokens=512
)

In [27]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,           # Step 4에서 만든 것
    chain_type="stuff",            # "stuff"는 검색 결과를 LLM에 다 넘김
    return_source_documents=True   # 🔍 근거 문서 포함 응답
)

In [35]:
query = "아 똥이 안나와"

response = qa_chain(query)

print("🧠 GPT 응답:")
print(response['result'])
# print("\n📎 참고 문서:")
# for doc in response['source_documents']:
#     print(doc.page_content)
#     print("-" * 60)

🧠 GPT 응답:
변비로 인해 배변이 어려운 상황이신가요? 변비가 지속되면 불편할 수 있으니 다음과 같은 방법을 시도해 보시는 것이 도움이 될 수 있습니다:

1. **수분 섭취 늘리기**: 물을 충분히 마시면 장운동이 원활해집니다.
2. **식이섬유 섭취**: 과일, 채소, 통곡물 등 식이섬유가 풍부한 음식을 드세요.
3. **규칙적인 운동**: 가벼운 산책이나 스트레칭이 장운동에 도움을 줄 수 있습니다.
4. **배변 습관 개선**: 규칙적인 시간에 화장실에 가는 습관을 들이세요.

만약 변비가 심하거나 통증, 출혈 등의 증상이 동반된다면, 빠르게 병원이나 의료 전문가와 상담하는 것이 중요합니다.

건강 보조제 관련해서는, 제공된 정보에 따르면 '티이아이-트리앙', '아이시안 듀얼액션', '우아닥터 면역젤리' 등이 있는데, 이들은 주로 면역 기능, 눈 건강, 항산화 작용 등에 도움을 주는 제품입니다. 변비 완화에 직접적인 도움을 주는 제품 정보는 포함되어 있지 않습니다.

변비가 계속된다면 전문의와 상담하시길 권장드립니다.
